In [11]:
# In this file 
# we train our proposed PolyVAE

import torch
import os
import numpy as np
from torch import optim
from torch.distributions import kl_divergence, Normal
from torch.nn import functional as F
from torch.optim.lr_scheduler import ExponentialLR
from model import PolyVAE_repara
from torch.utils.data import Dataset, DataLoader, TensorDataset

class MinExponentialLR(ExponentialLR):
    def __init__(self, optimizer, gamma, minimum, last_epoch=-1):
        self.min = minimum
        super(MinExponentialLR, self).__init__(optimizer, gamma, last_epoch=-1)

    def get_lr(self):
        return [
            max(base_lr * self.gamma**self.last_epoch, self.min)
            for base_lr in self.base_lrs
        ]
###############################
# initial parameters
s_dir = ""
batch_size = 64
n_epochs = 200
data_path = [s_dir + "data/poly_train.npy",
             s_dir + "data/poly_validate.npy",
             s_dir + "data/poly_train.npy"]
save_path = ""
lr = 1e-4
decay = 0.9999
hidden_dims = 512
z_dims = 1024
vae_beta = 0.1
input_dims = 130
seq_len = 10 * 16
beat_num = 10
tick_num = 16
save_period = 1
##############################


In [2]:
# input data
train_set = np.load(data_path[0], allow_pickle = True)
validate_set = np.load(data_path[1],allow_pickle = True) 

train_x = []
for i,data in enumerate(train_set):
    temp = []
    for d in data["layers"]:
        temp += d
    train_x.append(temp)
train_x = np.array(train_x)
# print(train_x.shape)

validate_x = []
for i,data in enumerate(validate_set):
    temp = []
    for d in data["layers"]:
        temp += d
    validate_x.append(temp)
validate_x = np.array(validate_x)
# print(train_x.shape)
train_x = torch.from_numpy(train_x).long()
validate_x = torch.from_numpy(validate_x).long()

print(train_x.size())
print(validate_x.size())

train_set = TensorDataset(train_x)
validate_set = TensorDataset(validate_x)

train_set = DataLoader(
    dataset = train_set,
    batch_size = batch_size, 
    shuffle = True, 
    num_workers = 8, 
    pin_memory = True, 
    drop_last = True
)
validate_set = DataLoader(
    dataset = validate_set,
    batch_size = batch_size, 
    shuffle = False, 
    num_workers = 8, 
    pin_memory = True, 
    drop_last = True
)


torch.Size([58041, 160])
torch.Size([7578, 160])


In [3]:
# import model
model = PolyVAE_repara(input_dims, hidden_dims, z_dims, seq_len, beat_num, tick_num, 4000)
optimizer = optim.Adam(model.parameters(), lr = lr)
if decay > 0:
    scheduler = MinExponentialLR(optimizer, gamma = decay, minimum = 1e-5)
if torch.cuda.is_available():
    print('Using: ', torch.cuda.get_device_name(torch.cuda.current_device()))
    model.cuda()
else:
    print('Using: CPU')


Using:  GeForce RTX 2080 SUPER


In [4]:
# process validete data from the dataloder
validate_data = []
for i,d in enumerate(validate_set):
    validate_data.append(d[0])
print(len(validate_data))

118


In [5]:
# loss function
def std_normal(shape):
    N = Normal(torch.zeros(shape), torch.ones(shape))
    if torch.cuda.is_available():
        N.loc = N.loc.cuda()
        N.scale = N.scale.cuda()
    return N

# def loss_function(recon, target, r_dis, beta):
#     CE = F.cross_entropy(recon.view(-1, recon.size(-1)), target, reduction = "mean")
# #     rhy_CE = F.nll_loss(recon_rhythm.view(-1, recon_rhythm.size(-1)), target_rhythm, reduction = "mean")
#     normal1 =  std_normal(r_dis.mean.size())
#     KLD1 = kl_divergence(r_dis, normal1).mean()
#     max_indices = recon.view(-1, recon.size(-1)).max(-1)[-1]
# #     print(max_indices)
#     correct = max_indices == target
#     acc = torch.sum(correct.float()) / target.size(0)
#     return acc, CE + beta * (KLD1)


In [6]:
def loss_function(recon, target, z_mean, z_std, beta):
    CE = F.cross_entropy(recon.view(-1, recon.size(-1)), target, reduction = "mean")
#     rhy_CE = F.nll_loss(recon_rhythm.view(-1, recon_rhythm.size(-1)), target_rhythm, reduction = "mean")
    
    # normal1 =  std_normal(r_dis.mean.size())
    # KLD1 = kl_divergence(r_dis, normal1).mean()
    mean_sq=z_mean * z_mean
    std_sq=z_std * z_std
    KLD1 = 0.5 * torch.mean(mean_sq + std_sq - torch.log(std_sq) - 1)

    max_indices = recon.view(-1, recon.size(-1)).max(-1)[-1]
#     print(max_indices)
    correct = max_indices == target
    acc = torch.sum(correct.float()) / target.size(0)
    return acc, CE + beta * (KLD1)

In [7]:
device = torch.device(torch.cuda.current_device())
device

device(type='cuda', index=0)

In [ ]:
# start training
logs = []
iteration = 0
step = 0
for epoch in range(n_epochs):
    print("epoch: %d\n__________________________________________" % (epoch), flush = True)
    mean_loss = 0.0
    mean_acc = 0.0
    v_mean_loss = 0.0
    v_mean_acc = 0.0
    total = 0
    for i, d in enumerate(train_set):
        # validate display
        x = gd = d[0]
        model.train()
        j = i % len(validate_data)
        v_x = v_gd = validate_data[j]
        
        x = x.to(device = device,non_blocking = True)
        gd = gd.to(device = device,non_blocking = True)
        v_x = v_x.to(device = device,non_blocking = True)
        v_gd = v_gd.to(device = device,non_blocking = True)
            
        optimizer.zero_grad()
        recon, r_dis, iteration, z_mu, z_var = model(x, gd)
        
        acc, loss = loss_function(recon, gd.view(-1), z_mu, z_var, vae_beta)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        mean_loss += loss.item()
        mean_acc += acc.item()
        
        model.eval()
        with torch.no_grad():
            v_recon, v_r_dis, _, v_z_mu, v_z_var = model(v_x, v_gd)
            v_acc, v_loss = loss_function(v_recon, v_gd.view(-1), v_z_mu, v_z_var, vae_beta)
            v_mean_loss += v_loss.item()
            v_mean_acc += v_acc.item()
        step += 1
        total += 1
        if decay > 0:
            scheduler.step()
        print("batch %d loss: %.5f acc: %.5f | val loss %.5f acc: %.5f iteration: %d"  
              % (i,loss.item(), acc.item(), v_loss.item(),v_acc.item(),iteration),flush = True)
    mean_loss /= total
    mean_acc /= total
    v_mean_loss /= total
    v_mean_acc /= total
    print("epoch %d loss: %.5f acc: %.5f | val loss %.5f acc: %.5f iteration: %d"  
              % (epoch, mean_loss, mean_acc, v_mean_loss, v_mean_acc, iteration),flush = True)
    logs.append([mean_loss,mean_acc,v_mean_loss,v_mean_acc,iteration])
    if (epoch + 1) % save_period == 0:
        filename = "reparavae-" + 'loss_' + str(mean_loss) + "_" + str(epoch+1) + "_" + str(iteration) + ".pt"
        torch.save(model.cpu().state_dict(),save_path + filename)
        model.cuda()
    np.save("reparavae-log.npy", logs)
        


epoch: 0
__________________________________________
batch 0 loss: 0.22510 acc: 0.94258 | val loss 0.84016 acc: 0.79248 iteration: 91507
batch 1 loss: 0.24129 acc: 0.94170 | val loss 0.88158 acc: 0.76846 iteration: 91508
batch 2 loss: 0.25343 acc: 0.93369 | val loss 0.52143 acc: 0.87461 iteration: 91509
batch 3 loss: 0.31150 acc: 0.91201 | val loss 0.20120 acc: 0.95283 iteration: 91510
batch 4 loss: 0.18436 acc: 0.95723 | val loss 0.55253 acc: 0.85723 iteration: 91511
batch 5 loss: 0.24582 acc: 0.93877 | val loss 0.72050 acc: 0.81211 iteration: 91512
batch 6 loss: 0.22055 acc: 0.94590 | val loss 0.30975 acc: 0.93018 iteration: 91513
batch 7 loss: 0.24953 acc: 0.93701 | val loss 0.47652 acc: 0.87441 iteration: 91514
batch 8 loss: 0.25770 acc: 0.93828 | val loss 0.56607 acc: 0.83799 iteration: 91515
batch 9 loss: 0.33335 acc: 0.91553 | val loss 0.43310 acc: 0.88809 iteration: 91516
batch 10 loss: 0.19166 acc: 0.95195 | val loss 0.29173 acc: 0.91934 iteration: 91517
batch 11 loss: 0.23452 

batch 96 loss: 0.25042 acc: 0.93848 | val loss 0.15440 acc: 0.96250 iteration: 91603
batch 97 loss: 0.21965 acc: 0.94268 | val loss 0.17801 acc: 0.95381 iteration: 91604
batch 98 loss: 0.28413 acc: 0.92949 | val loss 0.55963 acc: 0.85820 iteration: 91605
batch 99 loss: 0.24652 acc: 0.93545 | val loss 0.41843 acc: 0.89316 iteration: 91606
batch 100 loss: 0.27998 acc: 0.93057 | val loss 0.45232 acc: 0.88467 iteration: 91607
batch 101 loss: 0.19228 acc: 0.94941 | val loss 0.54773 acc: 0.86172 iteration: 91608
batch 102 loss: 0.23547 acc: 0.94023 | val loss 0.32524 acc: 0.91318 iteration: 91609
batch 103 loss: 0.21756 acc: 0.94746 | val loss 0.29725 acc: 0.92266 iteration: 91610
batch 104 loss: 0.21782 acc: 0.94229 | val loss 0.44897 acc: 0.87510 iteration: 91611
batch 105 loss: 0.22958 acc: 0.94355 | val loss 0.63113 acc: 0.86064 iteration: 91612
batch 106 loss: 0.20912 acc: 0.94902 | val loss 0.66230 acc: 0.86348 iteration: 91613
batch 107 loss: 0.18292 acc: 0.95537 | val loss 0.90572 ac

batch 192 loss: 0.26437 acc: 0.92959 | val loss 0.21878 acc: 0.93975 iteration: 91699
batch 193 loss: 0.24366 acc: 0.93545 | val loss 0.35595 acc: 0.90625 iteration: 91700
batch 194 loss: 0.21038 acc: 0.94971 | val loss 0.60247 acc: 0.87109 iteration: 91701
batch 195 loss: 0.25378 acc: 0.93760 | val loss 0.60717 acc: 0.87363 iteration: 91702
batch 196 loss: 0.24525 acc: 0.94053 | val loss 0.63288 acc: 0.83447 iteration: 91703
batch 197 loss: 0.22420 acc: 0.94531 | val loss 0.69069 acc: 0.78379 iteration: 91704
batch 198 loss: 0.21765 acc: 0.94707 | val loss 0.35120 acc: 0.90908 iteration: 91705
batch 199 loss: 0.20440 acc: 0.94814 | val loss 0.97345 acc: 0.75400 iteration: 91706
batch 200 loss: 0.26718 acc: 0.93350 | val loss 1.04546 acc: 0.74551 iteration: 91707
batch 201 loss: 0.22555 acc: 0.93789 | val loss 0.93639 acc: 0.75820 iteration: 91708
batch 202 loss: 0.16919 acc: 0.96064 | val loss 0.79157 acc: 0.78877 iteration: 91709
batch 203 loss: 0.30477 acc: 0.92324 | val loss 0.4090

batch 288 loss: 0.21954 acc: 0.94404 | val loss 0.84576 acc: 0.80752 iteration: 91795
batch 289 loss: 0.20537 acc: 0.95225 | val loss 0.61031 acc: 0.84688 iteration: 91796
batch 290 loss: 0.20529 acc: 0.94912 | val loss 0.43799 acc: 0.87490 iteration: 91797
batch 291 loss: 0.24442 acc: 0.93682 | val loss 0.40155 acc: 0.90156 iteration: 91798
batch 292 loss: 0.23589 acc: 0.94932 | val loss 0.37129 acc: 0.91074 iteration: 91799
batch 293 loss: 0.26851 acc: 0.92783 | val loss 0.72464 acc: 0.80957 iteration: 91800
batch 294 loss: 0.23037 acc: 0.94014 | val loss 0.46731 acc: 0.89756 iteration: 91801
batch 295 loss: 0.22848 acc: 0.94033 | val loss 0.19890 acc: 0.95430 iteration: 91802
batch 296 loss: 0.23519 acc: 0.94004 | val loss 0.27326 acc: 0.93105 iteration: 91803
batch 297 loss: 0.23115 acc: 0.94502 | val loss 0.29465 acc: 0.92617 iteration: 91804
batch 298 loss: 0.19113 acc: 0.95293 | val loss 0.56584 acc: 0.89434 iteration: 91805
batch 299 loss: 0.19764 acc: 0.95117 | val loss 0.4086

batch 384 loss: 0.23520 acc: 0.94365 | val loss 0.69546 acc: 0.85791 iteration: 91891
batch 385 loss: 0.25651 acc: 0.93711 | val loss 0.50486 acc: 0.88174 iteration: 91892
batch 386 loss: 0.29372 acc: 0.92148 | val loss 0.43782 acc: 0.87666 iteration: 91893
batch 387 loss: 0.18850 acc: 0.95762 | val loss 0.52170 acc: 0.86230 iteration: 91894
batch 388 loss: 0.23120 acc: 0.94102 | val loss 1.10877 acc: 0.69893 iteration: 91895
batch 389 loss: 0.27121 acc: 0.93076 | val loss 0.63904 acc: 0.84443 iteration: 91896
batch 390 loss: 0.24028 acc: 0.94004 | val loss 0.24241 acc: 0.93818 iteration: 91897
batch 391 loss: 0.21586 acc: 0.94561 | val loss 0.35234 acc: 0.91055 iteration: 91898
batch 392 loss: 0.21557 acc: 0.94541 | val loss 0.39302 acc: 0.89863 iteration: 91899
batch 393 loss: 0.26255 acc: 0.93330 | val loss 1.13926 acc: 0.71221 iteration: 91900
batch 394 loss: 0.24715 acc: 0.93945 | val loss 0.31956 acc: 0.90439 iteration: 91901
batch 395 loss: 0.19213 acc: 0.95625 | val loss 0.3879

...

batch 398 loss: 0.17150 acc: 0.96094 | val loss 0.40106 acc: 0.89473 iteration: 106401
batch 399 loss: 0.20523 acc: 0.95039 | val loss 0.31645 acc: 0.91982 iteration: 106402
batch 400 loss: 0.21714 acc: 0.95381 | val loss 0.50221 acc: 0.88223 iteration: 106403
batch 401 loss: 0.19011 acc: 0.95449 | val loss 0.38080 acc: 0.88623 iteration: 106404
batch 402 loss: 0.22750 acc: 0.94434 | val loss 0.18503 acc: 0.95186 iteration: 106405
batch 403 loss: 0.22411 acc: 0.94277 | val loss 0.20235 acc: 0.94297 iteration: 106406
batch 404 loss: 0.19029 acc: 0.95752 | val loss 0.12659 acc: 0.96973 iteration: 106407
batch 405 loss: 0.19883 acc: 0.95234 | val loss 0.79914 acc: 0.78477 iteration: 106408
batch 406 loss: 0.19355 acc: 0.95430 | val loss 0.88625 acc: 0.80088 iteration: 106409
batch 407 loss: 0.18039 acc: 0.95850 | val loss 0.59587 acc: 0.85439 iteration: 106410
batch 408 loss: 0.18644 acc: 0.95654 | val loss 0.38620 acc: 0.88877 iteration: 106411
batch 409 loss: 0.20011 acc: 0.95312 | val 

batch 493 loss: 0.23275 acc: 0.94209 | val loss 1.17599 acc: 0.74893 iteration: 106496
batch 494 loss: 0.22175 acc: 0.94561 | val loss 0.41272 acc: 0.92500 iteration: 106497
batch 495 loss: 0.18650 acc: 0.95508 | val loss 0.70368 acc: 0.82705 iteration: 106498
batch 496 loss: 0.17876 acc: 0.95947 | val loss 0.36406 acc: 0.90557 iteration: 106499
batch 497 loss: 0.21796 acc: 0.94463 | val loss 0.34301 acc: 0.92168 iteration: 106500
batch 498 loss: 0.19774 acc: 0.95215 | val loss 0.27424 acc: 0.94053 iteration: 106501
batch 499 loss: 0.17144 acc: 0.96455 | val loss 0.26776 acc: 0.93652 iteration: 106502
batch 500 loss: 0.22407 acc: 0.94375 | val loss 0.22536 acc: 0.93564 iteration: 106503
batch 501 loss: 0.22223 acc: 0.94219 | val loss 0.47601 acc: 0.87588 iteration: 106504
batch 502 loss: 0.23483 acc: 0.94102 | val loss 0.62602 acc: 0.86895 iteration: 106505
batch 503 loss: 0.23106 acc: 0.94463 | val loss 0.46606 acc: 0.88779 iteration: 106506
batch 504 loss: 0.19472 acc: 0.95684 | val 

batch 588 loss: 0.16968 acc: 0.96123 | val loss 0.37214 acc: 0.90967 iteration: 106591
batch 589 loss: 0.25721 acc: 0.93232 | val loss 0.71247 acc: 0.81924 iteration: 106592
batch 590 loss: 0.16222 acc: 0.96416 | val loss 0.88888 acc: 0.79023 iteration: 106593
batch 591 loss: 0.25622 acc: 0.93369 | val loss 0.81785 acc: 0.79697 iteration: 106594
batch 592 loss: 0.19598 acc: 0.94805 | val loss 0.50884 acc: 0.88066 iteration: 106595
batch 593 loss: 0.24025 acc: 0.94570 | val loss 0.20003 acc: 0.94873 iteration: 106596
batch 594 loss: 0.18663 acc: 0.95762 | val loss 0.59183 acc: 0.85420 iteration: 106597
batch 595 loss: 0.23010 acc: 0.94414 | val loss 0.70750 acc: 0.82314 iteration: 106598
batch 596 loss: 0.24532 acc: 0.93994 | val loss 0.31068 acc: 0.93535 iteration: 106599
batch 597 loss: 0.20161 acc: 0.95010 | val loss 0.39683 acc: 0.89668 iteration: 106600
batch 598 loss: 0.21983 acc: 0.94092 | val loss 0.56462 acc: 0.84727 iteration: 106601
batch 599 loss: 0.16223 acc: 0.96387 | val 

batch 683 loss: 0.26657 acc: 0.93662 | val loss 1.25624 acc: 0.71133 iteration: 106686
batch 684 loss: 0.20053 acc: 0.94922 | val loss 0.76769 acc: 0.78867 iteration: 106687
batch 685 loss: 0.22523 acc: 0.94697 | val loss 0.57357 acc: 0.82051 iteration: 106688
batch 686 loss: 0.20553 acc: 0.95020 | val loss 0.14235 acc: 0.96475 iteration: 106689
batch 687 loss: 0.16359 acc: 0.95869 | val loss 0.14887 acc: 0.96240 iteration: 106690
batch 688 loss: 0.22429 acc: 0.94580 | val loss 0.52343 acc: 0.87148 iteration: 106691
batch 689 loss: 0.29132 acc: 0.92568 | val loss 0.41425 acc: 0.89033 iteration: 106692
batch 690 loss: 0.21509 acc: 0.94912 | val loss 0.43755 acc: 0.89463 iteration: 106693
batch 691 loss: 0.21376 acc: 0.94844 | val loss 0.55725 acc: 0.86074 iteration: 106694
batch 692 loss: 0.19457 acc: 0.95312 | val loss 0.31238 acc: 0.92031 iteration: 106695
batch 693 loss: 0.20118 acc: 0.94922 | val loss 0.28665 acc: 0.92393 iteration: 106696
batch 694 loss: 0.20362 acc: 0.95176 | val 

batch 778 loss: 0.22227 acc: 0.94385 | val loss 0.28571 acc: 0.92900 iteration: 106781
batch 779 loss: 0.23192 acc: 0.94092 | val loss 0.32489 acc: 0.92100 iteration: 106782
batch 780 loss: 0.17620 acc: 0.95693 | val loss 0.38607 acc: 0.89785 iteration: 106783
batch 781 loss: 0.24113 acc: 0.93896 | val loss 0.21828 acc: 0.94365 iteration: 106784
batch 782 loss: 0.17762 acc: 0.95576 | val loss 0.19475 acc: 0.94678 iteration: 106785
batch 783 loss: 0.18823 acc: 0.95508 | val loss 0.35812 acc: 0.90635 iteration: 106786
batch 784 loss: 0.26217 acc: 0.93184 | val loss 0.60375 acc: 0.87158 iteration: 106787
batch 785 loss: 0.19899 acc: 0.95225 | val loss 0.57365 acc: 0.87910 iteration: 106788
